In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
import matplotlib.pyplot as plt
from multihead_diffatn import MultiheadDiffAttn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1337)

eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 50 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4

No fused RMSNorm


In [2]:
class args:
    def __init__(self, model_parallel_size, decoder_kv_attention_heads):
        self.model_parallel_size = model_parallel_size
        self.decoder_kv_attention_heads = decoder_kv_attention_heads

In [3]:
torch.manual_seed(1337)

test_args = args(1, None)
multihead = MultiheadDiffAttn(test_args, 200, 100,100)

v = torch.rand(1, 400,200)


print(multihead(v, 0))

tensor([[[ 0.1148, -0.1523, -0.1151,  ...,  0.1382, -0.1887,  0.1293],
         [ 0.0528, -0.1235, -0.1395,  ...,  0.0842, -0.1782,  0.0525],
         [ 0.0442, -0.1384, -0.1104,  ...,  0.0028, -0.0887,  0.1332],
         ...,
         [ 0.0292, -0.0828, -0.1127,  ...,  0.0275, -0.0417,  0.2730],
         [ 0.0387, -0.0851, -0.1072,  ...,  0.0216, -0.0468,  0.2674],
         [ 0.0349, -0.0808, -0.1091,  ...,  0.0201, -0.0469,  0.2683]]],
       grad_fn=<UnsafeViewBackward0>)


In [4]:
class positionFeedFoward(nn.Module):
    def __init__(self, inp, hid, drop=0.0):
        super(positionFeedFoward, self).__init__()
        self.w1 = nn.Linear(inp,4*hid)
        self.w2 = nn.Linear(4*hid,inp)
        self.drop = nn.Dropout(drop)

    def forward(self, x):

        x = self.w2(F.relu(self.w1(x)))
        x = self.drop(x)

        return x

In [5]:
class Decoder(nn.Module):
    '''Combinds MultiHeadedAttention and FeeForward, three layers'''
    def __init__(self, nheads, embn, con_l, drop=0.0):
        super(Decoder, self).__init__()
        head_size = embn // nheads
        #fix arguments

        self.slf_attn = MultiheadDiffAttn(args(1, None), embn, head_size, nheads)
        
        self.ffn = positionFeedFoward(embn, embn, drop=drop)

        self.norm1 = nn.LayerNorm(embn)
        self.norm2 = nn.LayerNorm(embn)

    def forward(self, x):
        x = x + self.slf_attn(self.norm1(x), 0)
        x = x + self.ffn(self.norm2(x))

        return x

In [6]:
#heads, d_model, d_km d_v as per the paper
enc = Decoder(8, 64, 512)
#batches, dims, dimensionalityxn_heads

v = torch.rand(1,512,64)


enc(v)

tensor([[[ 0.7053,  1.0463,  0.2766,  ...,  0.6525, -0.3046,  0.6850],
         [ 0.2670,  0.8096,  0.7059,  ...,  1.0027,  0.2972,  0.0287],
         [ 0.2243,  1.0756,  0.6185,  ...,  0.1017, -0.5243,  0.6314],
         ...,
         [ 1.2037,  0.2817,  1.0194,  ...,  0.1746,  0.8499,  0.9789],
         [ 0.3529,  0.8892,  1.2204,  ...,  0.7721,  0.0205,  0.1838],
         [ 1.1805,  0.7231, -0.0279,  ...,  0.6330,  0.0888,  0.5993]]],
       grad_fn=<AddBackward0>)

In [7]:
#Pytoch version adapted from here https://pub.aimind.so/creating-sinusoidal-positional-embedding-from-scratch-in-pytorch-98c49e153d6

class PosEncoding(nn.Module):
    def __init__(self, hid, n_pos=200):
        super(PosEncoding, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_pos, hid))

    def _get_sinusoid_encoding_table(self, n_pos, hid):

        if hid %2 != 0:
            raise ValueError("Sinusoidal positional embedding cannot apply to odd token embedding dim={}".format(hid))
        
        positions = torch.arange(0,n_pos).unsqueeze_(1)
        embeds = torch.zeros(n_pos, hid)

        denom = torch.pow(10000, 2 * torch.arange(0, hid//2)/2)
        embeds[:, 0::2] = torch.sin(positions/denom)
        embeds[:, 1::2] = torch.cos(positions/denom)
        embeds = embeds.unsqueeze(0)

        return embeds
    
    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()

In [8]:
class languageModel(nn.Module):
    '''Decoder model'''
    def __init__(
            self, n_vocab, embn, n_layers, n_head, dropout=0.2 , con_l=200
    ):
        super(languageModel, self).__init__()
        self.con_l = con_l
        self.word_emb = nn.Embedding(n_vocab, embn)
        self.pos_enc = nn.Embedding(con_l, embn)
        self.stack = nn.Sequential(
            *[Decoder( n_head, embn, con_l, drop=dropout) for _ in range(n_layers)]
        )
       
        self.layer_norm = nn.LayerNorm(embn)
        self.fc = nn.Linear(embn, n_vocab)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, tar=None):
        #batch, time
        B, T = x.shape

        tok = self.word_emb(x)
        pos = self.pos_enc(torch.arange(T, device=device))
        x = tok + pos
        x = self.stack(x)
        x = self.layer_norm(x)
        logits = self.fc(x)

        if tar is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            tar = tar.view(B*T)
            loss = F.cross_entropy(logits, tar)

        return logits, loss
    
    def generate(self, x, max_length):
        #x is a BxT array of in current context
        for _ in range(max_length):
            x_cond = x[:, -self.con_l:]
            logits, loss = self(x_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)

        return x
    
    

In [9]:

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)[0]
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)
            loss = F.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
model = languageModel(vocab_size,  384,6, 6, con_l=50
    )
m = model.to(device)
# print the number of parameters in the model
#print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
print(next(m.parameters()).is_cuda)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    #B, T, C = logits.shape
    #logits = logits.view(B*T, C)
    #targets = yb.view(B*T)
    #loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_length=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

True
step 0: train loss 4.3811, val loss 4.3752
step 500: train loss 1.7066, val loss 1.8771
step 1000: train loss 1.5281, val loss 1.7197
step 1500: train loss 1.4254, val loss 1.6362
step 2000: train loss 1.3697, val loss 1.6090
step 2500: train loss 1.3299, val loss 1.5765
step 3000: train loss 1.2921, val loss 1.5636
step 3500: train loss 1.2673, val loss 1.5817
step 4000: train loss 1.2330, val loss 1.5711
step 4500: train loss 1.2097, val loss 1.5719
step 4999: train loss 1.1834, val loss 1.5591

Have pill'd their tricks down too late toward hands;
I cries the public wit of same trition keys,
And I'll now be with she denial what:
'Tis like you a son to do minister
With soul sport of love, I do not become to
sit us.

BUCKINGHAM:
God my less, my nurse, and thy brother
And they dod proved. Away, boy!

Positeus:
Come I alm alarum, what you say?

BENVOLIO:
It is abroad? worth and with mine arm?

POLIXENES:
O, great Pompers;
For diere I trow hence this wonder do on my revel;
I would ha

In [ ]:
sinp = "wherefore art thou"
sinp = torch.tensor(encode(sinp)).unsqueeze(0).to(device)
print(decode(m.generate(sinp, max_length=500)[0].tolist()))